In [2]:
from io import open
import glob
import os
import unicodedata
import string

text = open('shakespeare.txt','rb').read().decode(encoding='utf-8')
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [3]:
print(text[:250]) # ceasar!!

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
#unique characters
vocab = sorted(set(text))
n_letters = len(vocab)
print(f'{len(vocab)} unique characters')

65 unique characters


In [5]:
import torch

#como vou usar uma embedding layer, não vou querer usar one-hot-encoding
# misses first character
def seqToTensor(line):
    letter_indexes = [vocab.index(line[li]) for li in range(1,len(line))]
    return torch.LongTensor(letter_indexes)

seqToTensor('First Citizen:')

tensor([47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10])

In [17]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

#testar depois sequencias disjuntas!

class TextDataset(Dataset):
    def __init__(self,text,sequence_length):
        self.text = text
        self.sequence_length = sequence_length
    
    def __len__(self):
        return len(self.text) - self.sequence_length
    
    def __getitem__(self,idx):
        sequence = self.text[idx:idx+self.sequence_length+1]
        return seqToTensor(sequence[:-1]),seqToTensor(sequence[1:])

In [18]:
seq_length = 100
dataset = TextDataset(text=text,sequence_length=seq_length)
dataloader = DataLoader(dataset,batch_size=64,shuffle=True)

In [16]:
print(next(iter(dataloader)))

[tensor([[30, 33, 15,  ..., 42,  1, 61],
        [52, 57,  0,  ..., 44, 47, 41],
        [53,  1, 42,  ..., 51,  6,  1],
        ...,
        [52, 42,  1,  ..., 43, 43, 54],
        [47, 52,  1,  ..., 49, 47, 57],
        [ 1, 61, 46,  ...,  1, 39, 44]]), tensor([[33, 15, 20,  ...,  1, 61, 43],
        [57,  0, 32,  ..., 47, 41, 43],
        [ 1, 42, 56,  ...,  6,  1, 51],
        ...,
        [42,  1, 58,  ..., 43, 54,  1],
        [52,  1, 58,  ..., 47, 57, 57],
        [61, 46, 53,  ..., 39, 44, 58]])]


In [ ]:
#embedding dimension
embedding_dim = 256
rnn_units = 1024
hidden_size = 256

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self,vocab_size, embedding_dim, hidden_size, rnn_units):
        super().__init__(self)

        self.emb = nn.Embedding(num_embeddings=vocab_size,embedding_dim=embedding_dim) #embedding layer
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_size, num_layers=rnn_units)
        self.dense = nn.Linear
